In [ ]:
# 1. Instalamos las librerías necesarias (si no están ya instaladas)
!pip install inference-sdk opencv-python-headless matplotlib

import base64
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from inference_sdk import InferenceHTTPClient

# --- CONFIGURACIÓN ---
CLIENT_URL = "https://serverless.roboflow.com"
API_KEY = "TSBgZ1xROUpAy1f23XyY"  # Tu API Key
WORKSPACE = "enricthings"
WORKFLOW_ID = "detect-count-and-visualize"

# --- 2. SUBIR IMAGEN DESDE EL NAVEGADOR ---
print("👇 Por favor, sube tu imagen usando el botón de abajo:")
uploaded = files.upload()

# Iteramos por si subes más de una imagen
for filename in uploaded.keys():
    print(f"\n✅ Imagen recibida: {filename}")
    
    # --- 3. CONECTAR Y EJECUTAR WORKFLOW ---
    client = InferenceHTTPClient(
        api_url=CLIENT_URL,
        api_key=API_KEY
    )

    print("Enviando a Roboflow... procesando...")

    try:
        # Ejecutamos el workflow con el archivo que acabamos de subir
        result = client.run_workflow(
            workspace_name=WORKSPACE,
            workflow_id=WORKFLOW_ID,
            images={
                "image": filename
            },
            use_cache=True
        )

        # --- 4. PROCESAR Y MOSTRAR RESULTADO ---
        found_image = False
        
        # Buscamos la imagen visualizada en la respuesta (Base64)
        if result and isinstance(result, list):
            for key, value in result[0].items():
                # Detectamos si el valor parece ser una imagen en base64
                if isinstance(value, str) and len(value) > 1000 and "base64" in value[:50]:
                    
                    # Limpiamos el encabezado si existe
                    if "," in value:
                        value = value.split(",")[1]
                    
                    # Decodificamos de Base64 a imagen
                    image_bytes = base64.b64decode(value)
                    image_array = np.frombuffer(image_bytes, dtype=np.uint8)
                    decoded_img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                    
                    # Convertir de BGR a RGB para matplotlib
                    final_img = cv2.cvtColor(decoded_img, cv2.COLOR_BGR2RGB)
                    
                    # Mostrar
                    plt.figure(figsize=(12, 10))
                    plt.imshow(final_img)
                    plt.axis('off')
                    plt.title(f"Resultado: {filename}")
                    plt.show()
                    
                    found_image = True
                    break
        
        if not found_image:
            print("⚠️ El workflow se ejecutó, pero no devolvió una imagen visual para dibujar.")
            print("Resultado crudo:", result)

    except Exception as e:
        print(f"❌ Error al procesar la imagen: {e}")